In [1]:
#importações das bibliotecas
import pandas as pd
import requests


## Campeonato brasileiro 2024

In [2]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2024_-_S%C3%A9rie_A')

# Informações do campeonato de 2024
tabelas2024 = pd.read_html(seriaA2024.text)
tabela_classificação2024 = tabelas2024[6]
tabela_jogos2024 = tabelas2024[7]

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\3065207342.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas2024 = pd.read_html(seriaA2024.text)


In [3]:
#criando uma lista com os nomes dos times
nomes_times_2024 = list(tabela_jogos2024["Casa \ Fora"])
#criando uma lista com as siglas dos times
sigla_times_2024 = list(tabela_jogos2024.columns)
sigla_times_2024.pop(0)

#criando um dicionário com a sigla dos times
apelidos_times_2024 = dict(zip(sigla_times_2024, nomes_times_2024))


In [8]:
# Ajustar a tabela de jogos
tabela_jogos2024_ajustada = tabela_jogos2024.set_index("Casa \ Fora")

# Desempilhar a tabela
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.unstack().reset_index()

# Renomear as colunas
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

# Função para ajustar o nome dos times
def ajustar_apelido(linha, apelido):
    time = linha["Fora"]
    return apelido[time]

# Ajustar o nome dos times aplicando a função
tabela_jogos2024_ajustada["Fora"] = tabela_jogos2024_ajustada.apply(ajustar_apelido, axis=1, args=(apelidos_times_2024,))
# Remover jogos entre o mesmo time
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Fora"] != tabela_jogos2024_ajustada["Casa"]]


In [18]:
tabela_jogos2024_ajustada["Placar"] = tabela_jogos2024_ajustada["Placar"].fillna("A jogar")

jogos_realizados = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = tabela_jogos2024_ajustada[~tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = jogos_faltantes.drop(columns=["Placar"])

# Separando a coluna Placar em gols_casa e gols_fora
jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)
jogos_realizados = jogos_realizados.drop(columns=["Placar"])

"""
# Media de gols feitos dentro de casa de um time
# Essa Tabela diz a media de gols que o time fez em casa e a media de gols que o time sofreu dentro de casa
media_gols_casa = jogos_realizados.groupby("Casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})

# Media de gols feitos e sofridos fora de casa de um time
media_gols_fora = jogos_realizados.groupby("Fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})
"""


C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\1598105354.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\1598105354.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)


'\n# Media de gols feitos dentro de casa de um time\n# Essa Tabela diz a media de gols que o time fez em casa e a media de gols que o time sofreu dentro de casa\nmedia_gols_casa = jogos_realizados.groupby("Casa").mean(numeric_only=True)\nmedia_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})\n\n# Media de gols feitos e sofridos fora de casa de um time\nmedia_gols_fora = jogos_realizados.groupby("Fora").mean(numeric_only=True)\nmedia_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})\n'

## Campeonato Brasileiro 2023

In [20]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2023_-_S%C3%A9rie_A')

tabelas2023 = pd.read_html(seriaA2024.text)
tabela_jogos = tabelas2023[6]

nomes_times = list(tabela_jogos["Casa \ Fora"])
sigla_times = list(tabela_jogos.columns)
sigla_times.pop(0)

apelidos_times = dict(zip(sigla_times, nomes_times))

tabela_jogos_ajustada = tabela_jogos.set_index("Casa \ Fora")
tabela_jogos_ajustada = tabela_jogos_ajustada.unstack().reset_index()
tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

tabela_jogos_ajustada["Fora"] = tabela_jogos_ajustada.apply(ajustar_apelido, axis=1, args=(apelidos_times,))
tabela_jogos_ajustada = tabela_jogos_ajustada[tabela_jogos_ajustada["Casa"] != tabela_jogos_ajustada["Fora"]]

tabela_jogos_ajustada["gols_casa"] = tabela_jogos_ajustada["Placar"].str.split("–").str[0].astype(int)
tabela_jogos_ajustada["gols_fora"] = tabela_jogos_ajustada["Placar"].str.split("–").str[1].astype(int)
tabela_jogos_ajustada = tabela_jogos_ajustada.drop(columns=["Placar"])  


"""
media_gols_casa_2023 = tabela_jogos_ajustada.groupby("Casa").mean(numeric_only=True)
media_gols_casa_2023 = media_gols_casa_2023.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})

media_gols_fora_2023 = tabela_jogos_ajustada.groupby("Fora").mean(numeric_only=True)
media_gols_fora_2023 = media_gols_fora_2023.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})

# Removendo os times rebaixados
media_gols_casa_2023 = media_gols_casa_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])
media_gols_fora_2023 = media_gols_fora_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])
"""

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\359129081.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas2023 = pd.read_html(seriaA2024.text)


'\nmedia_gols_casa_2023 = tabela_jogos_ajustada.groupby("Casa").mean(numeric_only=True)\nmedia_gols_casa_2023 = media_gols_casa_2023.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})\n\nmedia_gols_fora_2023 = tabela_jogos_ajustada.groupby("Fora").mean(numeric_only=True)\nmedia_gols_fora_2023 = media_gols_fora_2023.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})\n\n# Removendo os times rebaixados\nmedia_gols_casa_2023 = media_gols_casa_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])\nmedia_gols_fora_2023 = media_gols_fora_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])\n'

In [28]:
# Unindo as duas tabelas 2023 - 2024

tabela_jogos_total = pd.concat([tabela_jogos_ajustada, jogos_realizados], ignore_index=True)
media_gols_casa = tabela_jogos_total.groupby("Casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})
media_gols_fora = tabela_jogos_total.groupby("Fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})

media_gols_casa = media_gols_casa.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])
media_gols_fora = media_gols_fora.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])


display(media_gols_casa)

,gols feitos_casa,gols sofridos_casa
Casa,,
Athletico Paranaense,1.791667,0.875000
Atlético Goianiense,0.750000,2.000000
Atlético Mineiro,1.500000,0.818182
Bahia,1.478261,0.956522
Botafogo,1.913043,0.869565
Corinthians,1.291667,1.083333
Criciúma,1.500000,2.500000
Cruzeiro,1.000000,0.954545
Cuiabá,1.166667,1.166667


In [30]:
tabela_estatistica = media_gols_casa.merge(media_gols_fora, left_index=True, right_index=True)
tabela_estatistica = tabela_estatistica.reset_index()
tabela_estatistica = tabela_estatistica.rename(columns={"Casa": "Time"})
"""
tabela_estatistica_2023 = media_gols_casa_2023.merge(media_gols_fora_2023, left_index=True, right_index=True)
tabela_estatistica_2023 = tabela_estatistica_2023.reset_index()
tabela_estatistica_2023 = tabela_estatistica_2023.rename(columns={"Casa": "Time"})
"""

'\ntabela_estatistica_2023 = media_gols_casa_2023.merge(media_gols_fora_2023, left_index=True, right_index=True)\ntabela_estatistica_2023 = tabela_estatistica_2023.reset_index()\ntabela_estatistica_2023 = tabela_estatistica_2023.rename(columns={"Casa": "Time"})\n'

## Previsões

In [12]:
# Disitribuição de Poisson - Eventos independentes
time_casa = "Vitória"
time_fora = "Vasco da Gama"

lambda_casa = 
display(jogos_faltantes)




,Fora,Casa
1,Athletico Paranaense,Atlético Goianiense
2,Athletico Paranaense,Atlético Mineiro
3,Athletico Paranaense,Bahia
4,Athletico Paranaense,Botafogo
5,Athletico Paranaense,Corinthians
...,...,...
392,Vasco da Gama,Grêmio
393,Vasco da Gama,Internacional
394,Vasco da Gama,Juventude
397,Vasco da Gama,São Paulo
